In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from pandas import merge, concat, read_csv, DataFrame, Series, isnull, date_range, read_excel, MultiIndex, cut
from IPython.core.display import HTML
from datetime import datetime, timedelta
from collections import defaultdict

get_ipython().run_line_magic(u'matplotlib', u'inline')

sns.set(style='white')
sns.set_style('ticks')

In [38]:
from util import wise_multiplier
import copy

# compute risk

In [11]:
# read data

df_a = read_csv('./data/a.csv', index_col=0)
df_n = read_csv('./data/n.csv', index_col=0)
ln = list(df_a.index.copy())
df_n = df_n.reindex(ln)
df_a = df_a[ln]

In [36]:
def compute_risk(df_a, df_n, SEX):
    
    """
    MAIN FUNCTION TO COMPUTE RISK
    df_a: input mobility network (adj)
    df_n: pop and prev by constituency
    SEX: gender for which to compute risk
    
    returns:
        a dictionary with all risk measures (networks, marginal aggregates of both risk importation
        and risk exportations)
    
    3 types of risk: local, due to residents of same constituency. visitor (green in the paper) = residents of
    other constituencies coming in. travel (orange in paper) = travel of residents to other constituencies
    """
    
    dinver = {'M': 'F', 'F': 'M'}
    SEX_OTHER = dinver[SEX]
    
    A = df_a.values.copy()
    N = df_n.loc[df_a.index, 'POPULATION'].values.copy()
    P = df_n.loc[df_a.index, 'PREV_{}'.format(SEX_OTHER)].values.copy()

    Peff = np.dot(np.transpose(A), N*P) / np.dot(np.transpose(A), N)
    Neff = np.dot(np.transpose(A), N)
    
    Adiago = A.diagonal()

    L = N * (Adiago**2.) * P / Neff

    # V
    V = wise_multiplier(A, left=N*P, right=Adiago/Neff)
    mesk = np.diag_indices_from(V)
    V[mesk] = 0.
    V = DataFrame(V, index=df_a.index, columns=df_a.columns)

    # T
    T = wise_multiplier(np.transpose(A), left=Peff)
    mesk = np.diag_indices_from(T)
    T[mesk] = 0.
    T = DataFrame(T, index=df_a.index, columns=df_a.columns)
    
    # T+V
    Mob = DataFrame(T+V, index=df_a.index, columns=df_a.columns)
    
    # local and marginal V, T
    ds = DataFrame({'local': L, 'visitor': V.values.sum(axis=0), 'travel': T.values.sum(axis=0)},index=df_a.index)
    ds = ds.assign(**{'q': df_n['PREV_{}'.format(SEX_OTHER)]})  # opposite sex prevalence
    ds = ds.assign(**{'mobility': ds['visitor']+ds['travel']})
    ds = ds.assign(**{'total': ds['mobility']+ds['local']})
    ds = ds.assign(**{'visitor/mobility': ds['visitor']/ds['mobility'],
                      'travel/mobility': ds['travel']/ds['mobility'],
                     'visitor/total': ds['visitor']/ds['total'], 'travel/total': ds['travel']/ds['total'],
                     'local/total': ds['local']/ds['total'],
                     'mobility/total': ds['mobility']/ds['total']})
    
    # network of relative risks (relative on mobility risk)
    rel_mob = {}
    ek = ufu.wise_multiplier(V.values, right=1./ds['mobility'].values)
    ek = DataFrame(ek, index=V.index, columns=V.columns)
    rel_mob['visitor'] = ek.copy()
    ek = ufu.wise_multiplier(T.values, right=1./ds['mobility'].values)
    ek = DataFrame(ek, index=T.index, columns=T.columns)
    rel_mob['travel'] = ek.copy()
    ek = ufu.wise_multiplier(Mob.values, right=1./ds['mobility'].values)
    ek = DataFrame(ek, index=Mob.index, columns=Mob.columns)
    rel_mob['mobility'] = ek.copy()
    
    # network of relative risks (relative on total risk)
    rel_tot = {}
    ek = ufu.wise_multiplier(V.values, right=1./ds['total'].values)
    ek = DataFrame(ek, index=V.index, columns=V.columns)
    rel_tot['visitor'] = ek.copy()
    ek = ufu.wise_multiplier(T.values, right=1./ds['total'].values)
    ek = DataFrame(ek, index=T.index, columns=T.columns)
    rel_tot['travel'] = ek.copy()
    ek = ufu.wise_multiplier(Mob.values, right=1./ds['total'].values)
    ek = DataFrame(ek, index=Mob.index, columns=Mob.columns)
    rel_tot['mobility'] = ek.copy()
    
    abs_tot = {}
    abs_tot['visitor'] = V
    abs_tot['travel'] = T
    abs_tot['mobility'] = Mob
    
    # exportation
    for pap in ('visitor', 'travel', 'mobility'):
        J = rel_tot[pap].copy().values
        ds = ds.assign(**{'out {}'.format(pap): Series(J.sum(axis=1), index=df_a.index)})
        
        
        
    return {'marginal': copy.deepcopy(ds),
            'absolute': {'visitor': copy.deepcopy(V), 'travel': copy.deepcopy(T),
                         'mobility': copy.deepcopy(Mob)},
            'relative on mobility': copy.deepcopy(rel_mob),
            'relative on total': copy.deepcopy(rel_tot)
           }

In [39]:
compute_risk(df_a, df_n, 'M')['marginal'].to_csv('./comp/M_marginal.csv')